In [272]:
import numpy as np
from dependence import VineCopula

In [273]:
dim = 3
family = np.zeros((dim, dim), dtype=int)
for i in range(1, dim):
    for j in range(i):
        family[i, j] = 1

par1 = np.zeros((dim, dim))
for i in range(1, dim):
    for j in range(i):
        par1[i, j] = 0.5
        
par2 = np.zeros((dim, dim))
for i in range(1, dim):
    for j in range(i):
        par2[i, j] = 0.

In [274]:
structure = np.zeros((dim, dim), dtype=int)
for i in range(dim):
    structure[i, 0:i+1, ] = i + 1
        
structure

array([[1, 0, 0],
       [2, 2, 0],
       [3, 3, 3]])

In [275]:
rvine = VineCopula(structure, family, par1, par2)

In [276]:
def get_pair(dim, index):
    k = 0
    for i in range(1, dim):
        for j in range(i):
            if k == index:
                return [i+1, j+1]
            k+=1

In [277]:
dim = 4
n_pairs = dim * (dim - 1) / 2
pairs_ids = [1, 4]
remaining_pairs = range(1, n_pairs+1)
for pair in pairs_ids:
    remaining_pairs.remove(pair)

structure = np.zeros((dim, dim), dtype=int)

for k, pair_id in enumerate(pairs_ids):
    i, j = get_pair(dim, pair_id)
    if k < dim - 1:
        structure[k, k] = i
        structure[dim-1, k] = j
        if k == dim - 2:
            structure[dim-1, dim-1] = j
    else:
        for k1 in structure[-1, :]:
            print k1
print structure

[[3 0 0 0]
 [0 4 0 0]
 [0 0 0 0]
 [1 2 0 0]]


In [278]:
np.diag(structure)[:2]

array([3, 4])

In [279]:
len(np.unique(np.diag(structure)))

3

In [280]:
dim = 4
pairs_ids = [1, 4]

n_pairs = dim * (dim - 1) / 2


def is_vine_structure(matrix):
    dim = matrix.shape[0]
    u_matrix = np.triu(matrix, k=1)
    diag = np.diag(matrix)
    assert matrix.max() == dim, "Maximum should be the dimension: %d != %d" % (matrix.max(), dim)
    assert matrix.min() == 0, "Minimum should be 0: %d != %d" % (matrix.min(), dim)
    assert matrix.shape[0] == matrix.shape[1], "Matrix should be squared"
    assert u_matrix.sum() == 0, "Matrix should be lower triangular"
    assert len(np.unique(diag)) == dim, "Element should be uniques on the diagonal: %d != %d" % (len(np.unique(diag)), dim)
    for i in range(dim):
        column_i = matrix[i:, i]
        assert len(np.unique(column_i)) == dim - i, "Element should be unique for each column: %d != %d" % ( len(np.unique(column_i)), dim - i)
        for node in diag[:i]:
            assert node not in column_i, "Previous main node should not exist after"
    return matrix
structure = np.zeros((dim, dim))

for i in range(dim):
    structure[i, :i+1] = i + 1
    
print structure
is_vine_structure(structure)

[[ 1.  0.  0.  0.]
 [ 2.  2.  0.  0.]
 [ 3.  3.  3.  0.]
 [ 4.  4.  4.  4.]]


array([[ 1.,  0.,  0.,  0.],
       [ 2.,  2.,  0.,  0.],
       [ 3.,  3.,  3.,  0.],
       [ 4.,  4.,  4.,  4.]])

In [17]:
structure = np.asarray(
[
    [5, 0, 0, 0, 0],
    [3, 2, 0, 0, 0],
    [4, 3, 1, 0, 0],
    [2, 4, 3, 4, 0],
    [1, 1, 4, 3, 3]    
    
])
is_vine_structure(structure)

array([[5, 0, 0, 0, 0],
       [3, 2, 0, 0, 0],
       [4, 3, 1, 0, 0],
       [2, 4, 3, 4, 0],
       [1, 1, 4, 3, 3]])

In [18]:
import itertools

In [304]:
dim = 4
forced_pairs_ids = [3, 2, 1, 4]

n_forced_pairs = len(forced_pairs_ids)
n_pairs = dim*(dim-1)/2
assert n_forced_pairs <= n_pairs, "Not OK!"
assert max(forced_pairs_ids) < n_pairs, "Not OK!"
n_conditionned = range(dim - 1, 0, -1)
forced_pairs = np.asarray([get_pair(dim, pair_id) for pair_id in forced_pairs_ids])
remaining_pairs_ids = range(0, n_pairs)
for pair_id in forced_pairs_ids:
    remaining_pairs_ids.remove(pair_id)
remaining_pairs = np.asarray([get_pair(dim, pair_id) for pair_id in remaining_pairs_ids])

print('Vine dimension: %d' % dim)
print('Conditioning information:')
for i in range(dim-1):
    print("\t%d pairs with %d conditionned variables" % (n_conditionned[i], i))
print("Concerned pairs: {0}".format(forced_pairs.tolist()))
print("Remaining pairs: {0}".format(remaining_pairs.tolist()))

Vine dimension: 4
Conditioning information:
	3 pairs with 0 conditionned variables
	2 pairs with 1 conditionned variables
	1 pairs with 2 conditionned variables
Concerned pairs: [[4, 1], [3, 2], [3, 1], [4, 2]]
Remaining pairs: [[2, 1], [4, 3]]


In [305]:
combinations = list(itertools.product([1, -1], repeat=n_forced_pairs))
n_combinations = len(combinations)

all_good_pairs = []
for k, comb in enumerate(combinations):
    pairs_k = np.zeros(forced_pairs.shape)
    for i, ci in enumerate(comb):
        if ci == -1:
            pairs_k[i, :] = forced_pairs[i, ::-1]
        else:
            pairs_k[i, :] = forced_pairs[i, :]
    # The 1st indice appears only once        
    if len(np.unique(pairs_k[:, 0])) == n_forced_pairs:
        # THe 1st indice of the previous pairs should not appears in next columns
        itsok = True
        for i in range(len(pairs_k[:, 0])):
            if pairs_k[i, 0] in pairs_k[i+1:, :]:
                print 'nope'
                itsok = False
        if itsok:
            all_good_pairs.append(pairs_k)

nope
nope
nope
nope


In [306]:
p = 1
good_pairs = all_good_pairs[p]
print len(all_good_pairs)
good_pairs

IndexError: list index out of range

In [302]:
structure = np.zeros((dim, dim), dtype=int)
for i, pair in enumerate(good_pairs):
    structure[i, i] = pair[0]
    structure[dim-1, i] = pair[1]
structure

array([[4, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 1, 0],
       [1, 3, 3, 0]])

In [303]:
prev_ind = []
for i in range(dim):
    values_i = structure[i:, i]
    used_values_i = values_i[values_i != 0].tolist() + prev_ind
    remaining_i = range(1, dim + 1)
    for var in used_values_i:
        if (var in remaining_i):
            remaining_i.remove(var)
    values_i[values_i == 0] = remaining_i
    prev_ind.append(values_i[0])
is_vine_structure(structure)

array([[4, 0, 0, 0],
       [2, 2, 0, 0],
       [3, 1, 1, 0],
       [1, 3, 3, 3]])